# Building an Image Classification Model with Artificial Neural Network

CRISP-ML(Q) Methodology
*   Business and Data Understanding
*   Data Preparation (Data Engineering)
*   Model Building (Machine Learning)
*   Model Evaluation and Tuning
*   Deployment
*   Monitoring and Maintenance



# Stage1 : Business and Data Understanding

- The goal in this model building is to take an image of a handwritten single digit, and determine what that digit is. The project aims to evaluate the hand writtern numbers (0-9).

- Business Objectives: Maximize the ability to recognize hand writtern documents

- Constraints: Availability of data (More data will always give better accuracy)

# Stage1(a) : Data Collection

- The MNIST database of handwritten digits, available from this page, has a training set of 60,000 examples, and a test set of 10,000 examples. It is a subset of a larger set available from NIST. The digits have been size-normalized and centered in a fixed-size image. In our case, we have taken a subset of these data with training set of 5000 examples, and a test set of 1000 examples (http://yann.lecun.com/exdb/mnist/).

# Stage1(b) : Data Dictionary

- Each image is 28 pixels in height and 28 pixels in width, for a total of 784 pixels in total. Each pixel has a single pixel-value associated with it, indicating the lightness or darkness of that pixel, with higher numbers meaning darker. This pixel-value is an integer between 0 and 255, inclusive.

- The training data set, (train.csv), has 785 columns. The first column, called "label", is the digit that was drawn by the user. The rest of the columns contain the pixel-values of the associated image.



# Install/import required libraries

In [1]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install keras

In [3]:
from tensorflow.keras.layers import Dense
# Import necessary libraries for MLP and reshaping the data structres
# from keras.datasets import mnist
from tensorflow.keras.models import Sequential
import pandas as pd
import numpy as np
from keras.utils import np_utils
# from tensorflow.keras.layers import Dense
# from keras.models import Sequential
# from keras.layers import Dense
# from keras.layers import Dropout,Flatten

## Loading data from MySQL database

- The Train and Test datasets were stored in SQL database (Secondary data source) and using connectors, the data is being loaded in to python for model building process



In [4]:
pip install mysql

In [5]:
pip install mysql.connector

  Created wheel for mysql.connector: filename=mysql_connector-2.2.9-cp39-cp39-win_amd64.whl size=247961 sha256=4b3b0e2e2b7822bd36dfb913e905516621190c37b8c1b741117071e3ec2bd8bf
  Stored in directory: c:\users\dell\appdata\local\pip\cache\wheels\7b\14\39\5aad423666e827dfe9a1fbcd111ac17171e7c9865d570780ce
Successfully built mysql.connector



In [15]:
import mysql.connector as sql

mydb = sql.connect(
  host="localhost",
  user="root",
  password="Password",
  auth_plugin='mysql_native_password'
)
print(mydb)

ProgrammingError: 1045 (28000): Access denied for user 'root'@'localhost' (using password: YES)

In [ ]:
#Train Data
mycursor = mydb.cursor()
mycursor.execute("SELECT * FROM ann.train_sample")
myresult = mycursor.fetchall()

In [ ]:
num_fields = len(mycursor.description)
field_names = [i[0] for i in mycursor.description]
train = pd.DataFrame(myresult,columns = field_names)
train.head()

In [ ]:
#Test Data
mycursor = mydb.cursor()
mycursor.execute("SELECT * FROM ann.test_sample")
myresult = mycursor.fetchall()

In [ ]:
num_fields = len(mycursor.description)
field_names = [i[0] for i in mycursor.description]
test = pd.DataFrame(myresult,columns = field_names)
test.head()

## Stage 2 : Data Preparation (Data Engineering)

## Stage 2(a) : Exploratory Data Analysis

In [ ]:
new_train_f = train.drop(["label"],axis=1)

In [ ]:
new_train_f = new_train_f.values.reshape(-1,28,28,1)

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(new_train_f[2999][:,:,0])

In [ ]:
train

## Train and Test data - Data splitting

In [ ]:
# Separating the data set into 2 parts - all the inputs and label columns
# converting the integer type into float32 format 
x_train = train.iloc[:,1:].values.astype("float32")
x_test = test.iloc[:,1:].values.astype("float32")
y_train = train.label.values.astype("float32")
y_test = test.label.values.astype("float32")

## Data Normalization

In [ ]:
# Normalizing the inputs to fall under 0-1 by 
# diving the entire data with 255 (max pixel value)
x_train = x_train/255
x_test = x_test/255

In [ ]:
# one hot encoding outputs for both train and test data sets 
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

In [ ]:
# Storing the number of classes into the variable num_of_classes 
num_of_classes = y_test.shape[1]
x_train.shape
y_train.shape
x_test.shape
y_test.shape

## Stage 3 : Model Building (Machine Learning)

In [ ]:
# Creating a user defined function to return the model for which we are
# giving the input to train the ANN mode
def design_mlp():
    # Initializing the model 
    model = Sequential()
    model.add(Dense(150, input_dim =784, activation="relu"))
    model.add(Dense(200, activation="tanh"))
    model.add(Dense(100, activation="tanh"))
    
    model.add(Dense(num_of_classes, activation="softmax"))
    model.compile(loss="categorical_crossentropy", metrics=["accuracy"])
    return model

In [ ]:
# building a ANN model using train data set and validating on test data set
model = design_mlp()

# fitting model on train data
model.fit(x=x_train, y=y_train, batch_size=1000, epochs=10)
model.summary()

## Stage 4 : Model Evaluation and Tuning

In [ ]:
# Evaluating the model on test data  
eval_score_test = model.evaluate(x_test, y_test, verbose = 1)
print ("Accuracy: %.3f%%" %(eval_score_test[1]*100)) 
# accuracy on test data set

32/32 [==============================] - 0s 2ms/step - loss: 0.2831 - accuracy: 0.9160
Accuracy: 91.600%


In [ ]:
# accuracy score on train data 
eval_score_train = model.evaluate(x_train, y_train, verbose=0)
print ("Accuracy: %.3f%%" %(eval_score_train[1]*100)) 
# accuracy on train data set

Accuracy: 93.360%


## Stage 5 : Model Deployement

### Saving Model (Importing Pickel File)

In [ ]:
""" model_json = model.to_json()
with open("ImageClassifier.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("ImageClassifier.h5") """

In [ ]:
# import pickle
# pickle.dump(model, open('ann_model.pkl', 'wb'))

In [ ]:
# load the model from disk
# model = pickle.load(open('ann_model.pkl', 'rb'))

In [ ]:
# from flask import Flask
# app = Flask(__name__)
# @app.route('/')
# def home():
# 	return render_template('index.html')

# @app.route('/predict', methods=['GET','POST'])
# def predict():
#     '''
#     predict function to predict the image
#     Api hits this function when someone clicks submit.
#     '''
#     if request.method == 'POST':
#         # Get the image from post request
#         img = base64_to_pil(request.json)
        
#         # initialize model
#         model = get_ImageClassifierModel()

#         # Make prediction
#         preds = model_predict(img, model)

#         pred_proba = "{:.3f}".format(np.amax(preds))    # Max probability
#         pred_class = decode_predictions(preds, top=1)   # ImageNet Decode

#         result = str(pred_class[0][0][1])               # Convert to string
#         result = result.replace('_', ' ').capitalize()
        
#         # Serialize the result, you can add additional fields
#         return jsonify(result=result, probability=pred_proba)
#     return None
# #@app.route(‘/predict’, methods=[‘GET’, ‘POST’])

# def get_ImageClassifierModel():
#     model = model(weights='design_mlp()')
#     # Loading the pretrained model
#     model_json = open(Model_json, 'r')
#     loaded_model_json = model_json.read()
#     model_json.close()
#     model = model_from_json(loaded_model_json)
#     model.load_weights(Model_weigths)
#     return model
# if __name__ == '__main__':
#   app.run(host = '127.0.0.1',port = 5000)
  

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


In [ ]:
from flask import Flask, render_template, request, jsonify
import numpy as np
from tensorflow import keras
import cv2
import base64

# Initialize flask app
app = Flask(__name__)

# Load prebuilt model
model = keras.models.load_model('app/mnist_classification.h5')

# Handle GET request
@app.route('/', methods=['GET'])
def drawing():
    return render_template('drawing.html')

# Handle POST request
@app.route('/', methods=['POST'])
def canvas():
    # Recieve base64 data from the user form
    canvasdata = request.form['canvasimg']
    encoded_data = request.form['canvasimg'].split(',')[1]

    # Decode base64 image to python array
    nparr = np.fromstring(base64.b64decode(encoded_data), np.uint8)
    img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)

    # Convert 3 channel image (RGB) to 1 channel image (GRAY)
    gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Resize to (28, 28)
    gray_image = cv2.resize(gray_image, (28, 28), interpolation=cv2.INTER_LINEAR)

    # Expand to numpy array dimenstion to (1, 28, 28)
    img = np.expand_dims(gray_image, axis=0)

    try:
        prediction = np.argmax(model.predict(img))
        print(f"Prediction Result : {str(prediction)}")
        return render_template('drawing.html', response=str(prediction), canvasdata=canvasdata, success=True)
    except Exception as e:
        return render_template('drawing.html', response=str(e), canvasdata=canvasdata)


## Stage 6 : Monitoring and Maintanance 